In [2]:
import requests
import json
import time
from tqdm import tqdm
import KEY

In [33]:
fl_list = [
    "iati_identifier",
    "title_narrative",
    "reporting_org_narrative",
    "description_narrative_xml_lang",
    "description_narrative",
    "recipient_country_code",
    "recipient_region_code",
    "location_name_narrative",
    "activity_status_code",
    "document_link_url",
    "last_updated_datetime",
    "activity_date_type",
    "activity_date_iso_date",
    "activity_date_narrative",
    "sector_code",
    "sector_narrative",
    "sector_vocabulary",
    "tag_code",
    "tag_narrative",
    "tag_vocabulary",
    "policy_marker_code",
    "policy_marker_narrative",
    "policy_marker_code",
    "default_aid_type_code"
]

fl_str = ",".join(fl_list)

In [ ]:
url = "https://api.iatistandard.org/datastore/activity/select"
query = {
    "q": "*:*",
    "start": 0,
    "rows": 1000,
    "fl": "activity_id,title,description"
}

headers = {
    "Ocp-Apim-Subscription-Key": KEY.IATI_KEY
}

all_data = []  # Initialize an empty list to accumulate all data

try:
    total_rows = 0
    start_index = query["start"]
    rows_per_page = query["rows"]
    with tqdm(total=100, desc="Fetching Data") as pbar:
        while True:
            response = requests.get(url, params=query, headers=headers)
            if response.status_code != 200:
                print("Error:", response.status_code)
                break  # Exit the loop if there's an error
            data = response.json()
            num_found = data.get("response", {}).get("numFound", 0)
            total_rows += len(data.get("response", {}).get("docs", []))
            all_data.extend(data.get("response", {}).get("docs", []))  # Append data to the list

            if total_rows >= num_found:
                pbar.update(100 - pbar.n)  # Update progress bar to completion
                break  # Stop pagination if all documents are fetched
            else:
                progress = round(total_rows / num_found * 100, 2)
                pbar.update(progress - pbar.n)  # Update progress bar

            start_index += rows_per_page
            query["start"] = start_index

            # Sleep to respect rate limit
            time.sleep(0.1)

    print("All data fetched from api")

    # Dump all accumulated data to a single JSON file
    with open("response_data_all.json", "w") as file:
        json.dump(all_data, file, indent=4)  # Indent for readability
    print("Data downloaded and saved as 'response_data_all.json'")
except requests.exceptions.RequestException as e:
    print("Error:", e)

Fetching Data:  53%|█████▎    | 52.94/100 [26:22<23:26, 29.89s/it]   

Error: 403
All data fetched from api


Data downloaded and saved as 'response_data_all.json'


In [34]:
url = "https://api.iatistandard.org/datastore/activity/select"
query = {
    "q": "*:*",
    "start": 600,
    "rows": 2,  
    "fl": fl_str
}

headers = {
    "Ocp-Apim-Subscription-Key": "3a4048a6ae8a401f8111353112cc6fa5"
}

try:
    response = requests.get(url, params=query, headers=headers)
    if response.status_code == 200:
        data = response.json()
        # Überprüfen, ob Daten vorhanden sind
        docs = data.get("response", {}).get("docs", [])
        if docs:
            # Daten in Datei speichern
            with open("response_data.json", "w") as file:
                json.dump(docs, file, indent=4)
            print("Daten heruntergeladen und als 'response_data.json' gespeichert")
        else:
            print("Keine Daten in der Antwort gefunden.")
    else:
        print("Fehler:", response.status_code)
except requests.exceptions.RequestException as e:
    print("Fehler:", e)


Daten heruntergeladen und als 'response_data.json' gespeichert


In [35]:
import requests
import json
import csv

url = "https://api.iatistandard.org/datastore/activity/select"
query = {
    "q": "*:*",
    "start": 600,
    "rows": 2,
    "fl": fl_str
}

headers = {
    "Ocp-Apim-Subscription-Key": "3a4048a6ae8a401f8111353112cc6fa5"
}

fl_list = [
    "iati_identifier",
    "title_narrative",
    "reporting_org_narrative",
    "description_narrative_xml_lang",
    "description_narrative",
    "recipient_country_code",
    "recipient_region_code",
    "location_name_narrative",
    "activity_status_code",
    "document_link_url",
    "last_updated_datetime",
    "activity_date_type",
    "activity_date_iso_date",
    "sector_code",
    "sector_narrative",
    "sector_vocabulary",
    "tag_code",
    "tag_narrative",
    "tag_vocabulary",
    "policy_marker_code",
    "policy_marker_narrative",
    "policy_marker_code",
    "default_aid_type_code"
]

fl_str = ",".join(fl_list)

try:
    response = requests.get(url, params=query, headers=headers)
    if response.status_code == 200:
        data = response.json()
        # Überprüfen, ob Daten vorhanden sind
        docs = data.get("response", {}).get("docs", [])
        if docs:
            # Daten in Datei speichern
            with open("response_data.csv", mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fl_list)
                writer.writeheader()
                for doc in docs:
                    # Nur die Felder schreiben, die in fl_list enthalten sind
                    row = {key: doc.get(key, '') for key in fl_list}
                    writer.writerow(row)
            print("Daten heruntergeladen und als 'response_data.csv' gespeichert")
        else:
            print("Keine Daten in der Antwort gefunden.")
    else:
        print("Fehler:", response.status_code)
except requests.exceptions.RequestException as e:
    print("Fehler:", e)

Daten heruntergeladen und als 'response_data.csv' gespeichert
